# 01. Importing libraries and dataset

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Create path
path = r'/Users/Dulce/12-2023 Instacart Basket Analysis'

In [3]:
# Import customer.csv
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

## Eagle-view for Analysis

In [4]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [6]:
customers.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [7]:
customers.shape

(206209, 10)

# 03. Wrangle Data

## 3a. Data Type

In [8]:
## Change datatype for user_id
customers['user_id'] = customers['user_id'].astype('str')

In [9]:
## Check new datatype for user_id
customers['user_id'].dtype 

dtype('O')

## 3b. Rename columns

In [10]:
## Correct column name for "surname"
customers.rename(columns = {'Surnam' : 'Last Name'}, inplace = True)

In [11]:
## Correct column name for "state"
customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [12]:
## Correct column name for "fam_status"
customers.rename(columns = {'fam_status' : 'Marital_Status'}, inplace = True)

In [13]:
## Correct column name for "n_dependants"
customers.rename(columns = {'n_dependants' : 'Children'}, inplace = True)

## 3c. Drop unnecessary columns

In [14]:
## Drop column date_joined
customers = customers.drop(columns = ['date_joined'])

In [15]:
customers.head()

,user_id,First Name,Last Name,Gender,State,Age,Children,Marital_Status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1,married,40374


# 04. Data quality and Consistency checks

## 4a. Missing Values

In [16]:
# Finding missing values
customers.isnull().sum()

user_id               0
First Name        11259
Last Name             0
Gender                0
State                 0
Age                   0
Children              0
Marital_Status        0
income                0
dtype: int64

In [17]:
# Isolating null values for analysis
customers_nan = customers[customers['First Name'].isnull() == True]
customers_nan

,user_id,First Name,Last Name,Gender,State,Age,Children,Marital_Status,income
53,76659,NaN,Gilbert,Male,Colorado,26,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,3,married,151819
...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,0,single,56325


The 5% of the data for First Name is missing. I think it is not relevant information which would make impossible our analysis. I will keep them. For example, Marketing could use a simple "Ms./Mrs." in the cases which the first name is missing.

## 4b. Duplicates

In [18]:
customers_dups = customers[customers.duplicated()]
customers_dups

,user_id,First Name,Last Name,Gender,State,Age,Children,Marital_Status,income


No duplicates were found.

## 4c. Mixed Data Type

In [19]:
# Checking if Mixed Data Type exists
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

First Name


In [20]:
# Addressing Mixed Data Type in First Name Column
customers['First Name'] = customers['First Name'].astype('str')

In [21]:
## Check new datatype for First Name
customers['First Name'].dtype 

dtype('O')

# 05. Combine customers with ords_prods_merge_derived_flag.pkl

In [22]:
# Import ords_prods_merge_derived_flag
ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_derived_flag.pkl'))

In [23]:
# Change data type for user_id in ords_prods and delete the ".0"
ords_prods['user_id'] = ords_prods['user_id'].astype(str).str.replace('.0', '')

In [24]:
# Drop the existing indicator column in ords_prods
ords_prods = ords_prods.drop(columns = ['_merge'])

In [25]:
# Merge with indicator column
ords_prods_custs = ords_prods.merge(customers, on = 'user_id', indicator = True, how = 'inner')

In [26]:
ords_prods_custs.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,quantity,orders_day_of_week,...,frequency_flag,First Name,Last Name,Gender,State,Age,Children,Marital_Status,income,_merge
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,Frequent customer,Charles,Cox,Male,Minnesota,81,1,married,49620,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,Frequent customer,Charles,Cox,Male,Minnesota,81,1,married,49620,both
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,prior,1,5,...,Frequent customer,Charles,Cox,Male,Minnesota,81,1,married,49620,both
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,prior,10,5,...,Frequent customer,Charles,Cox,Male,Minnesota,81,1,married,49620,both
4,1000,Apricots,18,10,12.9,505689,138,prior,9,6,...,Frequent customer,Charles,Cox,Male,Minnesota,81,1,married,49620,both


In [27]:
# Checking the frequency of _merge column
ords_prods_custs['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

# 06. Export as ord_prod_cust.pkl

In [28]:
# Export data to pkl
ords_prods_custs.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_custs.pkl'))